# K-Means
### Definition
K-Means is an unsupervised learning algorithm that, as the name hints, finds a fixed number (k) of clusters in a set of data. A cluster is a group of data points that are grouped together due to similarities in their features. When using a K-Means algorithm, a cluster is defined by a centroid, which is a point (either imaginary or real) at the center of a cluster. Every point in a data set is part of the cluster whose centroid is most closely located. To put it simply, K-Means finds k number of centroids, and then assigns all data points to the closest cluster, with the aim of keeping the centroids small.

[(Alejandro and Sergio Villegas 2017)](https://blog.easysol.net/machine-learning-algorithms-3/)

### Algorithm
* Create two randomly generated centroids and assign each data point to the cluster of the closest centroid. 
* Assign objects to their closest cluster center according to the Euclidean distance function.
* Calculate the centroid or mean of all objects in each cluster.
* Repeat steps 2, 3 and 4 until the same points are assigned to each cluster in consecutive rounds.\


### Problem
* The initial centroids cannot be outlier
* How to choose the initial centroids should be discussed.
* How many cluster should choose should be discussed.

### Pros & Cons
* https://developers.google.com/machine-learning/clustering/algorithm/advantages-disadvantages

In [858]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
%matplotlib inline

In [859]:
# input data
# fist row means the first column feature, second row means the second column feature...
input_data = np.array([
    [12, 20, 28, 18, 29, 33, 24, 45, 45, 52, 51, 52, 55, 53, 55, 61, 64, 69, 72],
    [39, 36, 30, 52, 54, 46, 55, 59, 63, 70, 66, 63, 58, 23, 14, 8, 19, 7, 24],
    [32, 43, 35, 12, 74, 46, 54, 39, 13, 50, 79, 13, 38, 32, 41, 80, 29, 14, 14]
])
# how man group and how to choose first data point should be discussed
cluster_number = 3

# if the initial centroid is outlier, then the cluster number will be incorrect
def initial_centroid(initial_input, cluster_number):
    return np.random.randint(80, size=(cluster_number,initial_input.shape[0]))

initial_centroid = initial_centroid(input_data, cluster_number).reshape(-1, cluster_number)

In [860]:
def grouop_assignment(input_data, initial_centroids):
    # Calculate the distance (using Euclidean distance)
    input_data_transposeed = np.expand_dims(np.transpose(input_data), axis=0)
    initial_centroids = np.expand_dims(np.transpose(initial_centroids), axis=1)
    expended_centroids = initial_centroids
    expended_input = input_data_transposeed

    for i in range(cluster_number-1):
        expended_input = np.append(expended_input, input_data_transposeed, axis=0)
#     print('------expended_input------')
#     print(expended_input)
    
    for i in range(input_data_transposeed.shape[1]-1):
        expended_centroids = np.append(expended_centroids, initial_centroids, axis=1)  
#     print('------expended_centroids------')
#     print(expended_centroids)
    
    # Calculate the sum of the point and keep the dimension
    
    euclidean_distance = np.sqrt(np.sum(
                                np.square((expended_input - expended_centroids).reshape(-1, input_data.shape[0]))
                        , axis=1)).reshape(-1, input_data.shape[1],1)
#     print('------euclidean_distance-----')
#     print(euclidean_distance)
    # find closet centroids value
    cluster_index = np.argmin(np.stack(euclidean_distance, axis=1).reshape(-1, cluster_number), axis=1)
    return cluster_index
    


def update_centroids(input_data, cluster_index):
    # calculate new centroids by data in same group
    unique_cluster = np.unique(cluster_index)
    initial_input = np.expand_dims(np.transpose(input_data), axis=0)
    new_centroids = []

    for i in unique_cluster:
        index_of_same_group = np.where(cluster_index == i)
        same_group_data = np.take(initial_input, index_of_same_group, axis=1).reshape(-1, initial_input.shape[2])
        single_centroid = np.mean(same_group_data, axis=0)
        new_centroids = np.append(new_centroids, single_centroid, axis=0)

    new_centroids = new_centroids.reshape(-1, initial_input.shape[2]).reshape(-1, cluster_number)
    return new_centroids

cluster_index = grouop_assignment(input_data, initial_centroid)
new_centroids = update_centroids(input_data, cluster_index)
print(cluster_index)


[2 2 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]


In [861]:
cluster_index_laststep = []
result = []
while True:
    new_centroids = update_centroids(input_data, cluster_index)
    current_cluster_index = grouop_assignment(input_data, new_centroids)
    if np.array_equal(current_cluster_index, cluster_index_laststep):
        result = current_cluster_index
        break
    else:
        cluster_index_laststep = current_cluster_index

print(result)

[2 2 2 0 2 2 2 0 0 0 2 0 0 1 1 1 1 1 1]


In [862]:
from sklearn.cluster import KMeans
df = pd.DataFrame({
    'x': [12, 20, 28, 18, 29, 33, 24, 45, 45, 52, 51, 52, 55, 53, 55, 61, 64, 69, 72],
    'y': [39, 36, 30, 52, 54, 46, 55, 59, 63, 70, 66, 63, 58, 23, 14, 8, 19, 7, 24],
    'z': [32, 43, 35, 12, 74, 46, 54, 39, 13, 50, 79, 13, 38, 32, 41, 80, 29, 14, 14]
})
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_
print(labels)

[0 0 0 0 2 0 2 0 0 2 2 0 0 1 1 1 1 1 1]
